In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-DmqPWGZkTUndCRcQS3M-MGr2Yva5lCl' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-DmqPWGZkTUndCRcQS3M-MGr2Yva5lCl" -O Clarkson_Crossmatch.pth && rm -rf /tmp/cookies.txt

In [ ]:
!git clone https://github.com/keivanB/Clarkson_Finger_Gen

In [ ]:
# using Pytorch to run the converted weights 
#https://github.com/huangzh13/StyleGAN.pytorch

In [ ]:
!git clone https://github.com/huangzh13/StyleGAN.pytorch

In [ ]:
!pip install yacs

In [ ]:
import os
import sys
new_path = '/content/StyleGAN.pytorch'
sys.path.append(new_path)
import numpy as np
from tqdm import tqdm

import torch
from torchvision.utils import save_image

from models.GAN import Generator
from config import cfg as opt
import matplotlib.pyplot as plt

In [ ]:
def adjust_dynamic_range(data, drange_in=(-1, 1), drange_out=(0, 1)):
    """
    adjust the dynamic colour range of the given input data
    :param data: input image data
    :param drange_in: original range of input
    :param drange_out: required range of output
    :return: img => colour range adjusted images
    """
    if drange_in != drange_out:
        scale = (np.float32(drange_out[1]) - np.float32(drange_out[0])) / (
                np.float32(drange_in[1]) - np.float32(drange_in[0]))
        bias = (np.float32(drange_out[0]) - np.float32(drange_in[0]) * scale)
        data = data * scale + bias
    return torch.clamp(data, min=0, max=1)

In [ ]:
#inputs for Pytorch
Path_config = "/content/Clarkson_Finger_Gen/sample_clarkson.yaml"
Path_model = "/content/Clarkson_Crossmatch.pth"
Num_samples = 32 # number of sample that you want.
# Creating a folder for saving samples
save_path = '/content/Generated_Samples'
os.makedirs(save_path, exist_ok=True)

In [ ]:
opt.merge_from_file(Path_config)
opt.freeze()

In [ ]:
print("Creating generator object ...")
# create the generator object
gen = Generator(resolution=opt.dataset.resolution,
                num_channels=opt.dataset.channels,
                structure=opt.structure,
                **opt.model.gen)

print("Loading the generator weights from:", Path_model)
# load the weights into it
gen.load_state_dict(torch.load(Path_model))

In [ ]:
# path for saving the files:

latent_size = opt.model.gen.latent_size
out_depth = int(np.log2(opt.dataset.resolution)) - 2


print("Generating scale synchronized images ...")
for img_num in tqdm(range(1, Num_samples + 1)):
    # generate the images:
    with torch.no_grad():
        point = torch.randn(1, latent_size)
        point = (point / point.norm()) * (latent_size ** 0.5)
        ss_image = gen(point, depth=out_depth, alpha=1)
        # color adjust the generated image:
        ss_image = adjust_dynamic_range(ss_image)

    # save the ss_image in the directory
    save_image(ss_image, os.path.join(save_path, str(img_num) + ".png"))

    print("Generated %d images at %s" % (Num_samples, save_path))

In [ ]:
fig, ax = plt.subplots(8,4, figsize=(30,30))
for i in range(32):
    r = int(i%4)
    c = int(i/4)
    ax[c,r].imshow(plt.imread(os.path.join(save_path, str(i+1)+'.png')))